In [1]:
!ls
!pip install gradio

musc_det_CNN_79.73.h5  sample_data
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import math  # For mathematical calculations
import librosa  # For audio processing
import numpy as np  # For numerical operations
from tensorflow import keras  # For loading the trained model

# Load the pre-trained model for genre classification
model = keras.models.load_model("musc_det_CNN_79.73.h5");

# Function to preprocess the audio file
def process_input(audio_file, track_duration=30):
    SAMPLE_RATE = 22050  # Sample rate for audio processing
    NUM_MFCC = 13  # Number of MFCCs to extract
    N_FTT = 2048  # Number of samples per FFT
    HOP_LENGTH = 512  # Number of samples between successive frames
    TRACK_DURATION = track_duration  # Duration of the audio to process (in seconds)
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION  # Total samples in the track
    NUM_SEGMENTS = 10  # Number of segments to split the audio into

    # Calculate samples and MFCC vectors per segment
    samples_per_segment = int(SAMPLES_PER_TRACK / NUM_SEGMENTS)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / HOP_LENGTH)

    # Load the audio file and resample it
    signal, sample_rate = librosa.load(audio_file, sr=SAMPLE_RATE)

    # Extract MFCCs for the first segment
    start = samples_per_segment * 0  # Start index of the first segment
    finish = start + samples_per_segment  # End index of the first segment
    mfcc = librosa.feature.mfcc(
        y=signal[start:finish],
        sr=sample_rate,
        n_mfcc=NUM_MFCC,
        n_fft=N_FTT,
        hop_length=HOP_LENGTH,
    )

    # Transpose the MFCC array to make it suitable for the model
    mfcc = mfcc.T
    return mfcc

# Dictionary to map the model's predicted index to a genre
genre_dict = {
    0: "disco",
    1: "pop",
    2: "classical",
    3: "metal",
    4: "rock",
    5: "blues",
    6: "hiphop",
    7: "reggae",
    8: "country",
    9: "jazz",
}

# Function to predict the genre of the audio file
def predict_genre(audio_file):
    # Preprocess the uploaded audio file
    new_input_mfcc = process_input(audio_file, 30)
    # Reshape the input to match the model's expected input shape
    X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
    # Make the prediction using the loaded model
    prediction = model.predict(X_to_predict)
    # Get the genre index with the highest probability
    predicted_index = np.argmax(prediction, axis=1)
    # Map the predicted index to the genre name
    predicted_genre = genre_dict[int(predicted_index)]
    return f"Predicted Genre: {predicted_genre}"



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [3]:
#interface
import gradio as gr  # Import Gradio for building the UI
# Create the Gradio interface
interface = gr.Interface(
    fn=predict_genre,  # Function to be executed when user uploads a file
    inputs=gr.Audio(type="filepath"),  # Accept uploaded audio files
    outputs="text",  # Display the prediction as text
    title="Music Genre Classifier",  # Title of the app
    description="Upload an audio file to predict its genre.",  # Short description
)

# Launch the Gradio app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da2fb5dd222682be81.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
